In [2]:
from scapy.all import *
import os
print(os.sys.path)

['', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/home/ubuntu/.local/lib/python3.6/site-packages', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/usr/lib/python3/dist-packages/IPython/extensions', '/home/ubuntu/.ipython']


In [3]:
from scapy.layers.http import *

In [4]:
import pandas as pd
import numpy as np

In [5]:
import time
explore(scapy.layers.http)
timestamp = time.time()
print(timestamp)
local_time = time.localtime(1499084904.913831)
time.strftime('%Y-%m-%d-%H-%M-%S%f', local_time)

1570079473.7681015


'2017-07-03-12-28-24%f'

In [6]:
a = datetime.fromtimestamp(1499084904.913831)
print(a)
strftime('%A, %d/%m/%y, %H:%M:%S:%f', time.localtime(1499084904.913831))
"""
    #print('IP src_ip:', pkt[IP].src)
    #print('IP dst_id:', pkt[IP].dst)
    local_time = time.gmtime(pkt.time)
    #%H:%M:%S.%f
    local_time_transfered = time.strftime('%A, %d/%m/%y, %H:%M:%S:%f', local_time)
    print(str(pkt.time).split('.'))
    print(local_time_transfered)
    print(datetime.fromtimestamp(pkt.time))

    #print(pkt.flags)
    a = pkt.sprintf('%TCP.flags%')
    if (a == 'S'):
        #print("yes", a, pkt[IP].src, pkt[IP].dst)
        list_.append(pkt)
    """

2017-07-03 12:28:24.913831


'\n    #print(\'IP src_ip:\', pkt[IP].src)\n    #print(\'IP dst_id:\', pkt[IP].dst)\n    local_time = time.gmtime(pkt.time)\n    #%H:%M:%S.%f\n    local_time_transfered = time.strftime(\'%A, %d/%m/%y, %H:%M:%S:%f\', local_time)\n    print(str(pkt.time).split(\'.\'))\n    print(local_time_transfered)\n    print(datetime.fromtimestamp(pkt.time))\n\n    #print(pkt.flags)\n    a = pkt.sprintf(\'%TCP.flags%\')\n    if (a == \'S\'):\n        #print("yes", a, pkt[IP].src, pkt[IP].dst)\n        list_.append(pkt)\n    '

In [8]:
#del features
list_ = []
features = {"src_ip":[], "dst_ip":[], "time_stamp":[], "protocol":[], 'flag': [], 'src_port':[], 'dst_port':[]}

In [9]:
a = 0
b = 0
for index,pkt in enumerate(PcapReader('day2_1.pcap')):
    if IP in pkt:
        features['src_ip'].append(pkt[IP].src)
        features['dst_ip'].append(pkt[IP].dst)
        if TCP in pkt:
            features['protocol'].append('TCP')
            features['src_port'].append(pkt[TCP].sport)
            features['dst_port'].append(pkt[TCP].dport)
            features['flag'].append(pkt.sprintf('%TCP.flags%'))
        elif UDP in pkt:
            features['protocol'].append('UDP')
            features['src_port'].append(pkt[UDP].sport)
            features['dst_port'].append(pkt[UDP].dport)
            features['flag'].append('OTHER')
        elif ICMP in pkt:
            features['protocol'].append('ICMP')
            features['src_port'].append('OTHER')
            features['dst_port'].append('OTHER')
            features['flag'].append('OTHER')
        else:
            features['protocol'].append('OTHER')
            features['flag'].append('OTHER')
            features['src_port'].append('OTHER')
            features['dst_port'].append('OTHER')
        features["time_stamp"].append(pkt.time)


In [10]:
day_2_df = pd.DataFrame(features)
day_2_df.time_stamp = day_2_df.time_stamp.astype(np.float64)
#create this new feature to make it easy to group
day_2_df['src_to_dst_ip'] = day_2_df.src_ip +"->"+ day_2_df.dst_ip
# the standard deviation of time stamp between the messages between the same src_ip and dst_ip 
map_ = day_2_df.groupby(['src_to_dst_ip'])['time_stamp'].std()
day_2_df['time_std'] = day_2_df['src_to_dst_ip'].map(map_)
day_2_df.head()

,dst_ip,dst_port,flag,protocol,src_ip,src_port,time_stamp,src_to_dst_ip,time_std
0,192.168.10.14,51197,A,TCP,13.107.4.50,80,1.499085e+09,13.107.4.50->192.168.10.14,8.627729
1,13.107.4.50,80,A,TCP,192.168.10.14,51197,1.499085e+09,192.168.10.14->13.107.4.50,11.222483
2,192.168.10.14,51197,A,TCP,13.107.4.50,80,1.499085e+09,13.107.4.50->192.168.10.14,8.627729
3,13.107.4.50,80,A,TCP,192.168.10.14,51197,1.499085e+09,192.168.10.14->13.107.4.50,11.222483
4,192.168.10.14,51197,A,TCP,13.107.4.50,80,1.499085e+09,13.107.4.50->192.168.10.14,8.627729


In [11]:
#calculate the time interval between this message and its preceeding message between the same src_ip and dst_ip
for i in day_2_df['src_to_dst_ip'].value_counts().index:
    day_2_df.loc[day_2_df['src_to_dst_ip'] == i, 'time_interval'] = day_2_df[day_2_df['src_to_dst_ip'] == i].time_stamp.diff()

In [12]:
day_2_df.time_interval = day_2_df.time_interval.fillna(day_2_df.time_interval.mean())

In [13]:
day_2_df.head()

,dst_ip,dst_port,flag,protocol,src_ip,src_port,time_stamp,src_to_dst_ip,time_std,time_interval
0,192.168.10.14,51197,A,TCP,13.107.4.50,80,1.499085e+09,13.107.4.50->192.168.10.14,8.627729,0.213304
1,13.107.4.50,80,A,TCP,192.168.10.14,51197,1.499085e+09,192.168.10.14->13.107.4.50,11.222483,0.213304
2,192.168.10.14,51197,A,TCP,13.107.4.50,80,1.499085e+09,13.107.4.50->192.168.10.14,8.627729,0.000192
3,13.107.4.50,80,A,TCP,192.168.10.14,51197,1.499085e+09,192.168.10.14->13.107.4.50,11.222483,0.000330
4,192.168.10.14,51197,A,TCP,13.107.4.50,80,1.499085e+09,13.107.4.50->192.168.10.14,8.627729,0.000193


In [14]:
X = day_2_df[['dst_ip', 'flag', 'protocol', 'src_ip', 'time_interval', 'src_port', 'dst_port', 'time_std']]

In [15]:
X.loc[X.src_port == 'OTHER', 'src_port'] = -1
X.loc[X.dst_port == 'OTHER', 'dst_port'] = -1

/usr/lib/python3/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [16]:
X['src_port'] = X['src_port'].astype(int)
X['dst_port'] = X['dst_port'].astype(int)

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/lib/python3/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [17]:
# handel ip addresses 
# 1: internal ip
# 0: external ip
def search_ip_type(ip):
    ip_ = ip.split(".")
    if ip_[0] == '192' and ip_[1] == '168' and ip_[2] == '10':
        return 1
    return 0
X.dst_ip = X.dst_ip.apply(lambda x : search_ip_type(x))
X.src_ip = X.src_ip.apply(lambda x : search_ip_type(x))

/usr/lib/python3/dist-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [18]:
X.head()

,dst_ip,flag,protocol,src_ip,time_interval,src_port,dst_port,time_std
0,1,A,TCP,0,0.213304,80,51197,8.627729
1,0,A,TCP,1,0.213304,51197,80,11.222483
2,1,A,TCP,0,0.000192,80,51197,8.627729
3,0,A,TCP,1,0.000330,51197,80,11.222483
4,1,A,TCP,0,0.000193,80,51197,8.627729


In [19]:
# hanlde protocol: arbitrary encoding to decrease the dimension

print(X.protocol.value_counts())
X_encode = X.protocol.value_counts().reset_index()
X_encode['encode'] = range(X_encode.shape[0])

map_protocol = X_encode.set_index('index')['encode'] + 1
X.protocol = X.protocol.map(map_protocol)

TCP      780660
UDP       17281
ICMP         86
OTHER        29
Name: protocol, dtype: int64


/usr/lib/python3/dist-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [20]:
# handle flag: arbitrary encoding to decrease the dimension
print(X.flag.value_counts())
flag_map = {
    "S":1,
    "SA":2,
    "A":3
}
def flag_map_(x):
    if x in flag_map:
        return flag_map[x]
    else:
        return 4
X.flag = X.flag.apply(lambda x : flag_map_(x))

A        730977
PA        36887
OTHER     17396
FA         5920
S          3015
SA         3014
R           390
RA          376
FPA          81
Name: flag, dtype: int64


/usr/lib/python3/dist-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [21]:
# handle src_port and dst_port : frequency encoding
src_port_map = X.src_port.value_counts()
dst_port_map = X.dst_port.value_counts()
X.src_port = X.src_port.map(src_port_map)
X.dst_port = X.dst_port.map(dst_port_map)

/usr/lib/python3/dist-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [22]:
X

,dst_ip,flag,protocol,src_ip,time_interval,src_port,dst_port,time_std
0,1,3,1,0,0.213304,390646,100985,8.627729
1,0,3,1,1,0.213304,80088,300078,11.222483
2,1,3,1,0,0.000192,390646,100985,8.627729
3,0,3,1,1,0.000330,80088,300078,11.222483
4,1,3,1,0,0.000193,390646,100985,8.627729
5,1,3,1,0,0.000191,390646,100985,8.627729
6,1,3,1,0,0.000192,390646,100985,8.627729
7,0,3,1,1,0.000502,80088,300078,11.222483
8,1,3,1,0,0.000192,390646,100985,8.627729
9,1,3,1,0,0.000192,390646,100985,8.627729


In [23]:
X.fillna(X.mean(), inplace = True)

/usr/lib/python3/dist-packages/pandas/core/generic.py:4355: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [24]:
X.isnull().any().any()

False

In [25]:
X.to_csv("day2_1_processed.csv", index = False)

In [88]:
import pandas as pd
import numpy as np
X_total = pd.read_csv("day2_1_processed.csv").reset_index(drop = True)

In [2]:
X = X_total.sample(frac = 0.01)

In [91]:
X.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [18]:
X.reset_index(drop = True, inplace = True)

In [19]:
X

,dst_ip,flag,protocol,src_ip,time_interval,src_port,dst_port,time_std
0,0,3,1,1,0.000178,194578,300078,48.734366
1,1,4,1,0,0.000192,390646,259137,48.420823
2,1,4,2,1,0.001191,6393,11,89.002196
3,1,4,1,0,0.000193,390646,259137,48.420823
4,1,3,1,0,0.000193,390646,1122,3.639157
5,1,3,1,0,0.000227,390646,259137,48.420823
6,1,3,1,0,0.000192,390646,259137,48.420823
7,1,3,1,0,0.000458,390646,259137,48.420823
8,1,3,1,0,0.000191,390646,100985,8.627729
9,1,3,1,0,0.000193,390646,259137,48.420823


# DBSCAN ： V1

In [3]:
#the min_samples can be used to mitigate the effects of curse of high dimensionality in distance based clustering
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
X_Scaled = StandardScaler().fit_transform(X)
clustering = DBSCAN(eps=5, min_samples=16).fit(X_Scaled)

In [5]:
clustering

DBSCAN(algorithm='auto', eps=5, leaf_size=30, metric='euclidean',
       metric_params=None, min_samples=16, n_jobs=None, p=None)

In [9]:
cluster_labels = pd.Series(clustering.labels_)

In [ ]:
cluster_labels.value_counts()

In [34]:
X[cluster_labels == 1].protocol.value_counts()

2    162
Name: protocol, dtype: int64

In [37]:
cluster_cores = pd.Series(clustering.core_sample_indices_)

In [40]:
#percentgate of cores
cluster_cores.shape[0]/X.shape[0]

0.9971181556195965

# DBSCAN：V2

In [42]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
X_Scaled = StandardScaler().fit_transform(X)
clustering = DBSCAN(eps=3, min_samples=16).fit(X_Scaled)

In [43]:
cluster_labels = pd.Series(clustering.labels_)

In [44]:
cluster_labels.value_counts()

 0    7292
 1     442
 2     161
 4      31
 3      28
-1      27
dtype: int64

In [180]:
X[cluster_labels == 3]

,dst_ip,flag,protocol,src_ip,time_interval,src_port,dst_port,time_std
15,0,1,1,1,2.843917,5,42192,4.101985
30,0,1,1,1,0.000044,7,42192,26.759172
63,0,1,1,1,0.000100,14,300078,1.971873
207,0,1,1,1,0.300493,1,42192,0.369662
333,0,1,1,1,0.000489,9,42192,26.759172
352,0,1,1,1,0.006752,72,42192,26.759172
422,0,1,1,1,0.000259,9,42192,26.759172
659,0,1,1,1,0.005203,22,42192,45.179850
770,0,1,1,1,0.000071,7,42192,26.759172
1016,0,1,1,1,0.213304,23,300078,110.245203


In [83]:
cluster_cores = pd.Series(clustering.core_sample_indices_)

In [84]:
cluster_cores.shape[0]/X.shape[0]

0.9937351209121664

In [85]:
cluster_cores

0          0
1          1
2          2
3          3
4          4
5          5
6          6
7          7
8          8
9          9
10        10
11        11
12        12
13        13
14        14
15        15
16        16
17        17
18        18
19        19
20        20
21        21
22        22
23        23
24        24
25        25
26        26
27        27
28        28
29        29
        ... 
7901    7951
7902    7952
7903    7953
7904    7954
7905    7955
7906    7956
7907    7957
7908    7958
7909    7959
7910    7960
7911    7961
7912    7962
7913    7963
7914    7964
7915    7965
7916    7966
7917    7967
7918    7968
7919    7969
7920    7970
7921    7971
7922    7972
7923    7973
7924    7974
7925    7975
7926    7976
7927    7977
7928    7978
7929    7979
7930    7980
Length: 7931, dtype: int64

# K-means  ： V1

In [92]:
X_total.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [94]:
X_total.head()

,dst_ip,flag,protocol,src_ip,time_interval,src_port,dst_port,time_std
0,1,3,1,0,0.213304,390646,100985,8.627729
1,0,3,1,1,0.213304,80088,300078,11.222483
2,1,3,1,0,0.000192,390646,100985,8.627729
3,0,3,1,1,0.000330,80088,300078,11.222483
4,1,3,1,0,0.000193,390646,100985,8.627729


In [97]:
X_total.shape

(798056, 8)

In [119]:
from sklearn.cluster import KMeans
X_Scaled = StandardScaler().fit_transform(X_total)
kmeans = KMeans(n_clusters=2, random_state=0).fit(X_Scaled)
kmeans.labels_

array([0, 1, 0, ..., 1, 0, 1], dtype=int32)

In [120]:
kmeans_label = pd.Series(kmeans.labels_)

In [123]:
#cluser 2: From the result we can see that it is majorily clustered by the type of ip, both clusters are of compariable
#shape, one cluster is dst_ip == 1 and src_ip == 0, the other is src_ip == 1 and dst_ip == 1, now we change the number of
#clusters
X_total[kmeans_label == 1]

,dst_ip,flag,protocol,src_ip,time_interval,src_port,dst_port,time_std
1,0,3,1,1,0.213304,80088,300078,11.222483
3,0,3,1,1,0.000330,80088,300078,11.222483
7,0,3,1,1,0.000502,80088,300078,11.222483
11,0,3,1,1,0.000504,80088,300078,11.222483
14,0,3,1,1,0.000499,80088,300078,11.222483
18,0,3,1,1,0.000509,80088,300078,11.222483
21,0,3,1,1,0.000490,80088,300078,11.222483
23,0,3,1,1,0.000237,80088,300078,11.222483
26,0,3,1,1,0.000307,80088,300078,11.222483
28,0,3,1,1,0.000188,80088,300078,11.222483


In [126]:
X_total[kmeans_label == 1].src_ip.value_counts()

1    361992
0       348
Name: src_ip, dtype: int64

# K-means ： V2

In [167]:
from sklearn.cluster import KMeans
X_Scaled = StandardScaler().fit_transform(X_total)
kmeans = KMeans(n_clusters=5).fit(X_Scaled)
kmeans_label = pd.Series(kmeans.labels_)

In [169]:
kmeans_label.value_counts()

0    343487
2    262392
1    173493
3     16871
4      1813
dtype: int64

In [181]:
X_total[kmeans_label == 3]

,dst_ip,flag,protocol,src_ip,time_interval,src_port,dst_port,time_std
1522,0,4,2,1,2.133041e-01,937,937,124.543024
1523,0,4,2,1,2.133041e-01,937,937,135.046101
1525,0,4,2,1,2.133041e-01,937,937,129.197529
1526,0,4,2,1,2.133041e-01,937,937,128.029916
1527,0,4,2,1,2.133041e-01,937,937,122.627306
1528,0,4,2,1,2.133041e-01,937,937,123.460744
1732,1,4,2,0,2.133041e-01,937,937,135.045970
1781,1,4,2,0,2.133041e-01,937,937,122.625163
1789,1,4,2,0,2.133041e-01,937,937,128.027886
1794,1,4,2,0,2.133041e-01,937,937,124.542883


In [183]:
#https://www.sciencedirect.com/science/article/pii/S0031320319301608
#However, when the data has well separated clusters, the performance of k-means depends 
#completely on the goodness of the initialization. Therefore, if high clustering accuracy
#is needed, a better algorithm should be used instead.

# OCSVM

In [ ]:
from sklearn.svm import OneClassSVM
ocsvm = OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)

In [ ]:
ocsvm.fit(X_total)